In [10]:
import sys
from pathlib import Path

# 프로젝트 루트를 Python 경로에 추가 (rag 모듈 import를 위해 필요)
project_root = Path().resolve().parent  # 현재가 LangGraph 폴더이므로, parent가 프로젝트 루트
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ 프로젝트 루트 추가됨: {project_root}")

✅ 프로젝트 루트 추가됨: C:\Users\rkddn\YAI-NLP


컬렉션 정보 + 문서 목록

In [2]:
import chromadb

client = chromadb.PersistentClient(path="./chroma_db")

print("📂 컬렉션 목록:", [c.name for c in client.list_collections()])

collection = client.get_collection("my_collection")
print(f"📚 문서 수: {collection.count()}")

📂 컬렉션 목록: ['my_collection']
📚 문서 수: 182


저장된 문서 내용 보기

In [ ]:
result = collection.peek(limit=10)

results = collection.get(
    limit=10,
    include=["metadatas", "documents", "embeddings"],
    where={"source": "highmath12.txt"}
)

for i, (doc, meta) in enumerate(zip(results["documents"], results["metadatas"])):
    print(f"\n--- 청크 {i+1} ---")
    print(f"메타데이터: {meta}")
    print(f"내용: {doc[:200]}---")


--- 청크 1 ---
메타데이터: {'file_path': 'nlp.pdf', 'ModDate': "D:20231103094424+00'00'", 'page': 0, 'CreationDate': "D:20231103094424+00'00'", 'Producer': 'Skia/PDF m118', 'source': 'nlp.pdf', 'total_pages': 7, 'Creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'}
내용: 
 딥 러닝을 이용한 자연어 처리 입문 (/book/2155) / 15. 어텐션 메커니즘 (Attention … (/48948) / 15-01 어텐션 메커니즘 (Attentio… (/22893)
 도서 증정 이벤트 !! (https://wikidocs.net/219807) /  위키독스 (/)
15-01 어텐션 메커니즘 (Attention Mecha---

--- 청크 2 ---
메타데이터: {'page': 0, 'source': 'nlp.pdf', 'CreationDate': "D:20231103094424+00'00'", 'ModDate': "D:20231103094424+00'00'", 'Creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36', 'total_pages': 7, 'Producer': 'Skia/PDF m118', 'file_path': 'nlp.pdf'}
내용: 이 컨텍스트 벡터를 통해서 출력 시퀀스를 만들어냈습니다.
하지만 이러한 RNN에 기반한 seq2seq 모델에는 크게 두 가지 문제가 있습니다.
첫째, 하나의 고정된 크기의 벡터에 모든 정보를 압축하려고 하니까 정보 손실이 

특정 키워드로 검색 테스트

In [11]:
from rag.base import create_embedding_auto

embedding = create_embedding_auto()
query_vector = embedding.embed_query("수학1 핵심 커리큘럼을 설명해줘")

results = collection.query(
    query_embeddings=[query_vector],
    n_results=3,
    include=["metadatas", "documents", "distances"]
)

for i, (doc, meta, dist) in enumerate(zip(
    results["documents"][0], results["metadatas"][0], results["distances"][0]
)):
    print(f"\n--- 결과 {i+1} (거리: {dist:.4f}) ---")
    print(f"출처: {meta}")
    print(f"내용: {doc[:300]}")

🌐 임베딩: API 방식 (BAAI/bge-m3)

--- 결과 1 (거리: 0.6360) ---
출처: {'source': 'highmath12.txt'}
내용: 고등학교 교육과정의 핵심 축을 담당하는 **수학 I**과 **수학 II**는 수능 수학의 공통 과목이자, 이후 미적분이나 기하, 확률과 통계로 나아가는 필수적인 기초입니다. 요청하신 대로 텍스트로만 구성하여 각 과목의 핵심 개념과 주요 문제 유형을 상세히 정리해 드립니다.

---

### **[수학 I] : 함수와 수열의 해석**

수학 I은 크게 지수·로그함수, 삼각함수, 수열의 세 파트로 나뉩니다. 기존의 대수적 계산을 넘어 '새로운 함수의 그래프 해석'과 '규칙성의 발견'이 핵심입니다.

--- 결과 2 (거리: 0.6360) ---
출처: {'source': 'highmath12.txt'}
내용: 고등학교 교육과정의 핵심 축을 담당하는 **수학 I**과 **수학 II**는 수능 수학의 공통 과목이자, 이후 미적분이나 기하, 확률과 통계로 나아가는 필수적인 기초입니다. 요청하신 대로 텍스트로만 구성하여 각 과목의 핵심 개념과 주요 문제 유형을 상세히 정리해 드립니다.

---

### **[수학 I] : 함수와 수열의 해석**

수학 I은 크게 지수·로그함수, 삼각함수, 수열의 세 파트로 나뉩니다. 기존의 대수적 계산을 넘어 '새로운 함수의 그래프 해석'과 '규칙성의 발견'이 핵심입니다.

--- 결과 3 (거리: 0.8479) ---
출처: {'source': 'highmath12.txt'}
내용: ---

### **요약 및 학습 조언**

**수학 I**은 각 단원(지수로그, 삼각함수, 수열)이 독립적인 성격이 강해 특정 파트가 약하면 그 부분만 집중 공략하는 것이 가능합니다. 반면, **수학 II**는 극한 → 연속 → 미분 → 적분이 하나의 거대한 흐름으로 연결되어 있어 앞부분의 개념이 흔들리면 뒷부분(특히 적분)을 해결할 수 없습니다.


메타데이터 기준 필터링

In [16]:
# 특정 소스 파일의 문서만 보기
results = collection.get(
    where={"source": "nlp.pdf"},     # 메타데이터 필터
    include=["documents", "metadatas"]
)
print(f"nlp.pdf 청크 수: {len(results['documents'])}")

# 특정 페이지만 보기
results = collection.get(
    where={"page": 1},
    include=["documents", "metadatas"]
)

nlp.pdf 청크 수: 150


Summarization

In [18]:
# 컬렉션 전체 통계
all_data = collection.get(include=["metadatas"])
sources = set(m.get("source", "unknown") for m in all_data["metadatas"])

print(f"📊 컬렉션 요약")
print(f"  총 청크 수: {collection.count()}")
print(f"  소스 파일: {sources}")
for src in sources:
    count = sum(1 for m in all_data["metadatas"] if m.get("source") == src)
    print(f"    - {src}: {count}개 청크")

📊 컬렉션 요약
  총 청크 수: 182
  소스 파일: {'highmath12.txt', 'nlp.pdf'}
    - highmath12.txt: 32개 청크
    - nlp.pdf: 150개 청크
